# Who will survive in Titanic?

When the Titanic sank, 1502 of the 2224 passengers and crew were killed. One of the main reasons for this high level of casualties was the lack of lifeboats on this self-proclaimed "unsinkable" ship.

Those that have seen the movie know that some individuals were more likely to survive the sinking (lucky Rose) than others (poor Jack). Since we got the data about the passengers, it's a good idea to apply machine learning techniques to predict a passenger's chance of surviving using Python.

In [22]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import accuracy_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
# Load the train and test datasets to create two DataFrames
#train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv("train.csv")

#test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv("test.csv")
#Print the `head` of the train and test dataframes
#print(train.head())
#print(test.head())

.describe() summarizes the columns/features of the DataFrame, including the count of observations, mean, max and so on. Another useful trick is to look at the dimensions of the DataFrame. This is done by requesting the .shape attribute of your DataFrame object. (ex. your_data.shape)

In [8]:
train.describe()

/Users/xuemingli/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000         NaN    0.000000   
50%     446.000000    0.000000    3.000000         NaN    0.000000   
75%     668.500000    1.000000    3.000000         NaN    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [12]:
test.describe()

PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  332.000000  418.000000  418.000000  417.000000
mean   1100.500000    2.265550   30.272590    0.447368    0.392344   35.627188
std     120.810458    0.841838   14.181209    0.896760    0.981429   55.907576
min     892.000000    1.000000    0.170000    0.000000    0.000000    0.000000
25%     996.250000    1.000000         NaN    0.000000    0.000000         NaN
50%    1100.500000    3.000000         NaN    0.000000    0.000000         NaN
75%    1204.750000    3.000000         NaN    1.000000    0.000000         NaN
max    1309.000000    3.000000   76.000000    8.000000    9.000000  512.329200

In [14]:
train.shape
#train["Age"].count()

714

In [13]:
test.shape

(418, 11)

The training set has 891 observations and 12 variables, count for Age is 714.

In [15]:
# Passengers that survived vs passengers that passed away
print (train["Survived"].value_counts()), "\n"
# As proportions
print (train["Survived"].value_counts(normalize= True))

0    549
1    342
Name: Survived, dtype: int64 

0    0.616162
1    0.383838
Name: Survived, dtype: float64


In [18]:
# Males that survived vs males that passed away
print(train['Survived'][train['Sex']=='male'].value_counts())

# Normalized male survival
print(train['Survived'][train['Sex']=='male'].value_counts(normalize = True))

0    468
1    109
Name: Survived, dtype: int64
0    0.811092
1    0.188908
Name: Survived, dtype: float64


In [19]:
# Females that survived vs Females that passed away
print(train['Survived'][train['Sex']=='female'].value_counts())

# Normalized female survival
print(train['Survived'][train['Sex']=='female'].value_counts(normalize = True))

1    233
0     81
Name: Survived, dtype: int64
1    0.742038
0    0.257962
Name: Survived, dtype: float64


In [20]:
# Create the column Child and assign to 'NaN'
train["Child"] = float('NaN')
train["Child"][train["Age"]<18] = 1
train["Child"][train["Age"]>= 18] = 0
# Assign 1 to passengers under 18, 0 to those 18 or older. Print the new column.
print (train['Child'])

# Print normalized Survival Rates for passengers under 18
print(train["Survived"][train["Child"] == 1].value_counts(normalize = True))

# Print normalized Survival Rates for passengers 18 or older

print(train["Survived"][train["Child"] == 0].value_counts(normalize = True))


0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      NaN
6      0.0
7      1.0
8      0.0
9      1.0
10     1.0
11     0.0
12     0.0
13     0.0
14     1.0
15     0.0
16     1.0
17     NaN
18     0.0
19     NaN
20     0.0
21     0.0
22     1.0
23     0.0
24     1.0
25     0.0
26     NaN
27     0.0
28     NaN
29     NaN
      ... 
861    0.0
862    0.0
863    NaN
864    0.0
865    0.0
866    0.0
867    0.0
868    NaN
869    1.0
870    0.0
871    0.0
872    0.0
873    0.0
874    0.0
875    1.0
876    0.0
877    0.0
878    NaN
879    0.0
880    0.0
881    0.0
882    0.0
883    0.0
884    0.0
885    0.0
886    0.0
887    0.0
888    NaN
889    0.0
890    0.0
Name: Child, dtype: float64
1    0.539823
0    0.460177
Name: Survived, dtype: float64
0    0.618968
1    0.381032
Name: Survived, dtype: float64


/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
print train["Age"].median()
train["Age"] = train["Age"].fillna(train["Age"].median())

28.0


In [32]:
print train["Embarked"].value_counts()
# Impute the Embarked variable
train["Embarked"] = train['Embarked'].fillna('S')

# Convert the Embarked classes to integer form
try:
    train["Embarked"][train["Embarked"] == "S"] = 0
    train["Embarked"][train["Embarked"] == "C"] = 1
    train["Embarked"][train["Embarked"] == "Q"] = 2
print train["Embarked"]

SyntaxError: invalid syntax (<ipython-input-32-1ed8ee776a04>, line 10)

In [21]:
# Create a copy of test: test_one
test_one = test
test_one['Survived'] =0
test_one['Survived'][test_one['Sex']=='female'] = 1
print (test_one['Survived'])

# Initialize a Survived column to 0


# Set Survived to 1 if Sex equals "female" and print the `Survived` column from `test_one`

0      0
1      1
2      0
3      0
4      1
5      0
6      1
7      0
8      1
9      0
10     0
11     0
12     1
13     0
14     1
15     1
16     0
17     0
18     1
19     1
20     0
21     0
22     1
23     0
24     1
25     0
26     1
27     0
28     0
29     0
      ..
388    0
389    0
390    0
391    1
392    0
393    0
394    0
395    1
396    0
397    1
398    0
399    0
400    1
401    0
402    1
403    0
404    0
405    0
406    0
407    0
408    1
409    1
410    1
411    1
412    1
413    0
414    1
415    0
416    0
417    0
Name: Survived, dtype: int64


/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
# Convert the male and female groups to integer form
train["Sex"][train["Sex"] == "male"] = 0
train['Sex'][train['Sex'] == 'female'] = 1
print (train['Sex'])

0      0
1      1
2      1
3      1
4      0
5      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     1
15     1
16     0
17     0
18     1
19     1
20     0
21     0
22     1
23     0
24     1
25     1
26     0
27     0
28     1
29     0
      ..
861    0
862    1
863    1
864    0
865    1
866    1
867    0
868    0
869    0
870    0
871    1
872    0
873    0
874    1
875    1
876    0
877    0
878    0
879    1
880    1
881    0
882    1
883    0
884    0
885    1
886    0
887    1
888    1
889    0
890    0
Name: Sex, dtype: object


/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


**target** = train["Survived"].values

**features** = train[["Sex", "Age"]].values

**my_tree** = tree.DecisionTreeClassifier()

**my_tree** = my_tree.fit(features, target)

In [35]:
# Print the train data to see the available features
print(train.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris   0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...   1  38.0      1      0   
2                             Heikkinen, Miss. Laina   1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)   1  35.0      1      0   
4                           Allen, Mr. William Henry   0  35.0      0      0   

             Ticket     Fare Cabin  Embarked  Child  
0         A/5 21171   7.2500   NaN         0    0.0  
1          PC 17599  71.2833   C85         1    0.0  
2  STON/O2. 3101282   7.9250   NaN         0    0.0  
3            113803  53.1000  C123         0    0.0  
4            373450   8.0500   N

## Creating decision tree

In [59]:
# Create the target and features numpy arrays: target, features_one
target = train['Survived'].values
features_one = train[["Pclass", "Sex", "Age", "Fare"]].values

# Fit your first decision tree: my_tree_one
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)

# Look at the importance and score of the included features
print("Importances: "  "Pclass", my_tree_one.feature_importances_[0], "Sex", my_tree_one.feature_importances_[1], "Age", my_tree_one.feature_importances_[2], "Fare", my_tree_one.feature_importances_[3])
print(my_tree_one.score(features_one, target))

('Importances: Pclass', 0.12566688047907745, 'Sex', 0.31274009040120893, 'Age', 0.22475351379885414, 'Fare', 0.33683951532085937)
0.977553310887


In [62]:
# Impute the missing value with the median
test = pd.read_csv("test.csv")
print test.head()
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Sex"][test["Sex"] == "male"] = 0
test["Sex"][test["Sex"] == "female"] = 1
test["Fare"] = test["Fare"].fillna(test["Fare"].median())
print test.describe()

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  
       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  418.000000  418.000000  418.000000  418.000000
mean   1100.500000 

/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [83]:
# Extract the features from the test set: Pclass, Sex, Age, and Fare.
test_features = test[['Pclass', 'Sex', 'Age', 'Fare']].values

# Make your prediction using the test set
my_prediction = my_tree_one.predict(test_features)

# Create a data frame with two columns: PassengerId & Survived. Survived contains your predictions
PassengerId =np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
print(my_solution)

# Check that your data frame has 418 entries
print(my_solution.shape)

# Write your solution to a csv file with the name my_solution.csv
my_solution.to_csv("my_solution_one.csv", index_label = ["PassengerId"])
print(my_solution)

      Survived
892          0
893          0
894          1
895          1
896          1
897          0
898          0
899          0
900          1
901          0
902          0
903          0
904          1
905          1
906          1
907          1
908          0
909          1
910          1
911          0
912          0
913          1
914          1
915          0
916          1
917          0
918          1
919          1
920          1
921          0
...        ...
1280         0
1281         0
1282         0
1283         1
1284         0
1285         0
1286         0
1287         1
1288         0
1289         1
1290         0
1291         0
1292         1
1293         0
1294         1
1295         0
1296         0
1297         0
1298         0
1299         0
1300         1
1301         1
1302         1
1303         1
1304         0
1305         0
1306         1
1307         0
1308         0
1309         0

[418 rows x 1 columns]
(418, 1)
      Survived
892          0
893    

**Your submission scored 0.72249**

### Control the Overfitting

In DecisionTreeRegressor, the depth of our model is defined by two parameters: - the max_depth parameter determines when the splitting up of the decision tree stops. - the min_samples_split parameter monitors the amount of observations in a bucket. If a certain threshold is not reached (e.g minimum 10 passengers) no further splitting can be done.

In [105]:
# Create a new array with the added features: features_two
features_two = train[["Pclass","Age","Sex","Fare", 'SibSp', 'Parch', 'Embarked']].values
#print (features_two)

#Control overfitting by setting "max_depth" to 10 and "min_samples_split" to 5 : my_tree_two
max_depth = 10
min_samples_split = 5

my_tree_two = tree.DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split, random_state = 1)
my_tree_two = my_tree_two.fit(features_two, target)

#Print the score of the new decison tree
print my_tree_two.feature_importances_
print (my_tree_two.score(features_two,target))

[ 0.14130255  0.17906027  0.41616727  0.17938711  0.05039699  0.01923751
  0.0144483 ]
0.905723905724


In [87]:
test["Embarked"] = test['Embarked'].fillna('S')

# Convert the Embarked classes to integer form

test["Embarked"][test["Embarked"] == "S"] = 0
test["Embarked"][test["Embarked"] == "C"] = 1
test["Embarked"][test["Embarked"] == "Q"] = 2


/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/xuemingli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [104]:
# Extract the features from the test set: Pclass, Sex, Age, and Fare.
test_features = test[["Pclass","Age","Sex","Fare", 'SibSp', 'Parch', 'Embarked']].values

print test_features
# Make your prediction using the test set
my_prediction = my_tree_two.predict(test_features)
# Create a data frame with two columns: PassengerId & Survived. Survived contains your predictions
PassengerId =np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
print(my_solution)

# Check that your data frame has 418 entries
print(my_solution.shape)

# Write your solution to a csv file with the name my_solution.csv
my_solution.to_csv("my_solution_two.csv", index_label = ["PassengerId"])
print(my_solution)


[[3 34.5 0 ..., 0 0 2]
 [3 47.0 1 ..., 1 0 0]
 [2 62.0 0 ..., 0 0 2]
 ..., 
 [3 38.5 0 ..., 0 0 0]
 [3 27.0 0 ..., 0 0 0]
 [3 27.0 0 ..., 1 1 1]]
      Survived
892          0
893          0
894          0
895          1
896          1
897          0
898          0
899          0
900          1
901          0
902          1
903          0
904          1
905          0
906          1
907          1
908          0
909          0
910          1
911          0
912          1
913          1
914          1
915          0
916          1
917          0
918          1
919          0
920          1
921          0
...        ...
1280         0
1281         0
1282         0
1283         1
1284         1
1285         0
1286         0
1287         1
1288         0
1289         1
1290         0
1291         0
1292         1
1293         0
1294         1
1295         1
1296         0
1297         0
1298         0
1299         0
1300         1
1301         0
1302         1
1303         1
1304         0

**Your submission scored 0.76077**

## Feature-engineering##

A valid assumption is that larger families need more time to get together on a sinking ship, and hence have lower probability of surviving. Family size is determined by the variables SibSp and Parch, which indicate the number of family members a certain passenger is traveling with. So when doing feature engineering, you add a new variable family_size, which is the sum of SibSp and Parch plus one (the observation itself), to the test and train set.

In [90]:
# Create train_two with the newly defined feature
train_two = train.copy()
train_two["family_size"] = train['SibSp'] + train['Parch']+1

# Create a new feature set and add the new feature
features_three = train_two[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", 'family_size']].values

# Define the tree classifier, then fit the model
my_tree_three = tree.DecisionTreeClassifier()
my_tree_three = my_tree_three.fit(features_three, target)

# Print the score of this decision tree
print(my_tree_three.score(features_three, target))

0.979797979798


In [92]:
# Extract the features from the test set: Pclass, Sex, Age, and Fare.
test_two = test.copy()
test_two["family_size"] = test['SibSp'] + test['Parch']+1
test_features = test_two[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", 'family_size']].values
# Make your prediction using the test set
my_prediction = my_tree_three.predict(test_features)
# Create a data frame with two columns: PassengerId & Survived. Survived contains your predictions
PassengerId =np.array(test_two["PassengerId"]).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
print(my_solution)

# Check that your data frame has 418 entries
print(my_solution.shape)

# Write your solution to a csv file with the name my_solution.csv
my_solution.to_csv("my_solution_three.csv", index_label = ["PassengerId"])
print(my_solution)

      Survived
892          0
893          0
894          1
895          1
896          0
897          0
898          0
899          0
900          1
901          0
902          0
903          0
904          1
905          1
906          1
907          1
908          0
909          1
910          1
911          0
912          1
913          1
914          1
915          0
916          0
917          0
918          1
919          1
920          1
921          1
...        ...
1280         0
1281         0
1282         0
1283         1
1284         1
1285         0
1286         1
1287         1
1288         0
1289         1
1290         0
1291         0
1292         1
1293         0
1294         1
1295         1
1296         0
1297         0
1298         0
1299         0
1300         1
1301         1
1302         1
1303         1
1304         1
1305         0
1306         1
1307         0
1308         0
1309         0

[418 rows x 1 columns]
(418, 1)
      Survived
892          0
893    

**Your submission scored 0.62679**

## Random Forest 
 
Random Forest technique handles the overfitting problem you faced with decision trees. It grows multiple (very deep) classification trees using the training set. At the time of prediction, each tree is used to come up with a prediction and every outcome is counted as a vote. For example, if you have trained 3 trees with 2 saying a passenger in the test set will survive and 1 says he will not, the passenger will be classified as a survivor. 

In [93]:
# Import the `RandomForestClassifier`
from sklearn.ensemble import RandomForestClassifier

In [101]:
# We want the Pclass, Age, Sex, Fare,SibSp, Parch, and Embarked variables
features_forest = train[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values

# Building and fitting my_forest
forest = RandomForestClassifier(max_depth = 10, min_samples_split=2, n_estimators = 100, random_state = 1)
my_forest = forest.fit(features_forest, target)

# Print the score of the fitted random forest
print(my_forest.score(features_forest, target))

# Compute predictions on our test set features then print the length of the prediction vector
test_features = test[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values
pred_forest = my_forest.predict(test_features)
PassengerId =np.array(test_two["PassengerId"]).astype(int)
my_solution = pd.DataFrame(pred_forest, PassengerId, columns = ["Survived"])
print "Check that your data frame has 418 entries"
print(my_solution.shape)
# Write your solution to a csv file with the name my_solution.csv
my_solution.to_csv("my_solution_four.csv", index_label = ["PassengerId"])
print my_solution

0.933782267116
Check that your data frame has 418 entries
(418, 1)
      Survived
892          0
893          0
894          1
895          0
896          0
897          0
898          0
899          0
900          1
901          0
902          0
903          0
904          1
905          0
906          1
907          1
908          0
909          0
910          1
911          0
912          0
913          0
914          1
915          0
916          1
917          0
918          1
919          0
920          1
921          0
...        ...
1280         0
1281         0
1282         0
1283         1
1284         0
1285         0
1286         0
1287         1
1288         0
1289         1
1290         0
1291         0
1292         1
1293         0
1294         1
1295         0
1296         0
1297         0
1298         0
1299         0
1300         1
1301         1
1302         1
1303         1
1304         1
1305         0
1306         1
1307         0
1308         0
1309         0

[4

In [102]:
#Request and print the `.feature_importances_` attribute
print(my_tree_two.feature_importances_)
print(my_forest.feature_importances_)

#Compute and print the mean accuracy score for both models
print(my_tree_two.score(features_two, target))
print(my_forest.score(features_forest, target))

[ 0.14130255  0.17906027  0.41616727  0.17938711  0.05039699  0.01923751
  0.0144483 ]
[ 0.12007156  0.25298913  0.32877722  0.29816209]
0.905723905724
0.933782267116


The most important feature was "Sex", but it was more significant for "my_tree_two"

**Your submission scored 0.75598**

In [108]:
from sklearn.svm import SVC
# We want the Pclass, Age, Sex, Fare,SibSp, Parch, and Embarked variables
features_svc = train[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values

# Building and fitting my_forest
clf = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
my_pred = clf.fit(features_svc, target)

# Print the score of the fitted random forest
print(my_pred.score(features_svc, target))

# Compute predictions on our test set features then print the length of the prediction vector
tests_svc = test[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values
pred_svc = clf.predict(tests_svc)
PassengerId =np.array(test_two["PassengerId"]).astype(int)
my_solution = pd.DataFrame(pred_svc, PassengerId, columns = ["Survived"])
print "Check that your data frame has 418 entries"
print(my_solution.shape)
# Write your solution to a csv file with the name my_solution.csv
my_solution.to_csv("my_solution_svc.csv", index_label = ["PassengerId"])
print my_solution

#Compute and print the mean accuracy score for both models
print(clf.score(features_two, target))


0.892255892256
Check that your data frame has 418 entries
(418, 1)
      Survived
892          0
893          0
894          1
895          0
896          0
897          0
898          0
899          1
900          0
901          1
902          0
903          0
904          1
905          0
906          0
907          1
908          0
909          0
910          0
911          0
912          1
913          1
914          1
915          1
916          1
917          0
918          1
919          0
920          1
921          0
...        ...
1280         0
1281         0
1282         1
1283         1
1284         1
1285         0
1286         0
1287         1
1288         0
1289         1
1290         0
1291         0
1292         1
1293         1
1294         1
1295         1
1296         0
1297         0
1298         0
1299         1
1300         1
1301         1
1302         1
1303         1
1304         0
1305         0
1306         1
1307         0
1308         0
1309         0

[4